In [5]:
from __future__ import division

import matplotlib.pyplot as plt
import numpy.matlib as matlib
from scipy.stats import multivariate_normal
import numpy as np
# import support_code

### Support Code

In [7]:
def generateData(dataSize, noiseParams, actual_weights):
    # x1: from [0,1) to [-1,1)
    x1 = -1 + 2 * np.random.rand(dataSize, 1)
    # appending the bias term
    xtrain = np.matrix(np.c_[np.ones((dataSize, 1)), x1])
    # random noise
    noise = np.matrix(np.random.normal(
                            noiseParams["mean"],
                            noiseParams["var"],
                            (dataSize, 1)))

    ytrain = (xtrain * actual_weights) + noise

    return xtrain, ytrain

In [8]:
def make_plots(actual_weights, xtrain, ytrain, likelihood_var, prior, likelihoodFunc, getPosteriorParams, getPredictiveParams):

    # #setup for plotting
    #
    showProgressTillDataRows = [1, 2, 10, -1]
    numRows = 1 + len(showProgressTillDataRows)
    numCols = 4
    plt.figure(figsize=(10,10))
    plt.subplots_adjust(hspace=.8, wspace=.8)

    plotWithoutSeeingData(prior, numRows, numCols)

    # see data for as many rounds as specified and plot
    for roundNum, rowNum in enumerate(showProgressTillDataRows):
        current_row = roundNum + 1
        first_column_pos = (current_row * numCols) + 1

        # #plot likelihood on latest point
        plt.subplot(numRows, numCols, first_column_pos)


        likelihoodFunc_with_data = lambda W: likelihoodFunc(W,
                                                      xtrain[:rowNum,],
                                                      ytrain[:rowNum],
                                                      likelihood_var)
        contourPlot(likelihoodFunc_with_data, actual_weights)

        # plot updated posterior on points seen till now
        x_seen = xtrain[:rowNum]
        y_seen = ytrain[:rowNum]
        mu, cov = getPosteriorParams(x_seen, y_seen,
                                      prior, likelihood_var)
        posteriorDistr = multivariate_normal(mu.T.tolist()[0], cov)
        posteriorFunc = lambda x: posteriorDistr.pdf(x)
        plt.subplot(numRows, numCols, first_column_pos + 1)
        contourPlot(posteriorFunc, actual_weights)

        # plot lines
        dataSeen = np.c_[x_seen[:, 1], y_seen]
        plt.subplot(numRows, numCols, first_column_pos + 2)
        plotSampleLines(mu, cov, dataPoints=dataSeen)

        # plot predictive
        plt.subplot(numRows, numCols, first_column_pos + 3)
        postMean, postVar = getPosteriorParams(x_seen, y_seen, prior)
        plotPredictiveDistribution(getPredictiveParams, postMean, postVar)

    # #show the final plot
    plt.show()

In [9]:
def plotWithoutSeeingData(prior, numRows, numCols):

    #Blank likelihood
    plt.subplot(numRows, numCols, 1, axisbg='grey')
    plt.title("Likelihood")
    plt.xlabel("")
    plt.ylabel("")
    plt.xticks([])
    plt.yticks([])
    plt.xlim([-0.9, 0.9])
    plt.ylim([-0.9, 0.9])

    #Prior
    priorDistribution = multivariate_normal(mean=prior["mean"].T.tolist()[0],
        cov=prior["var"])
    priorFunc = lambda x:priorDistribution.pdf(x)
    plt.subplot(numRows, numCols, 2)
    plt.title("Prior/Posterior")
    contourPlot(priorFunc)

    # Plot initially valid lines (no data seen)
    plt.subplot(numRows, numCols, 3)
    plt.title("Data Space")
    plotSampleLines(prior["mean"], prior["var"])

    # Blank predictive
    plt.subplot(numRows, numCols, 4, axisbg='grey')
    plt.title('Predictive Distribution')
    plt.xticks([])
    plt.yticks([])
    plt.xlim([-1, 1])
    plt.ylim([-1, 1])
    plt.xlabel("")
    plt.ylabel("")

In [10]:
def contourPlot(distributionFunc, actualWeights=[]):

    stepSize = 0.05
    array = np.arange(-1, 1, stepSize)
    x, y_train = np.meshgrid(array, array)

    length = x.shape[0] * x.shape[1]
    x_flat = x.reshape((length, 1))
    y_flat = y_train.reshape((length, 1))
    contourPoints = np.c_[x_flat, y_flat]

    values = map(distributionFunc, contourPoints)
    values = np.array(values).reshape(x.shape)

    plt.contourf(x, y_train, values)
    plt.xlabel("w1")
    plt.ylabel("w2")
    plt.xticks([-0.5, 0, 0.5])
    plt.yticks([-0.5, 0, 0.5])
    plt.xlim([-0.9, 0.9])
    plt.ylim([-0.9, 0.9])

    if(len(actualWeights) == 2):
        plt.plot(float(actualWeights[0]), float(actualWeights[1]),
                 "*k", ms=5)



In [11]:
# Plot the specified number of lines of the form y_train = w0 + w1*x in [-1,1]x[-1,1] by
# drawing w0, w1 from a bivariate normal distribution with specified values
# for mu = mean and sigma = covariance Matrix. Also plot the data points as
# circles.
def plotSampleLines(mean, variance,
                    numberOfLines=6,
                    dataPoints=np.empty((0, 0))):
    stepSize = 0.05
    # generate and plot lines
    for round in range(1, numberOfLines):
        weights = np.matrix(np.random.multivariate_normal(mean.T.tolist()[0], variance)).T
        x1 = np.arange(-1, 1, stepSize)
        x = np.matrix(np.c_[np.ones((len(x1), 1)), x1])
        y_train = x * weights

        plt.plot(x1, y_train)

    # markings
    plt.xticks([-1, 0, 1])
    plt.yticks([-1, 0, 1])
    plt.xlim([-1, 1])
    plt.ylim([-1, 1])
    plt.xlabel("x")
    plt.ylabel("y")

    # plot data points if given
    if(dataPoints.size):
        plt.plot(dataPoints[:, 0], dataPoints[:, 1],
                 "co")

In [12]:
def plotPredictiveDistribution(getPredictiveParams,postMean, postVar):
    stepSize = 0.05
    x = np.arange(-1, 1, stepSize)
    x = np.matrix(np.c_[np.ones((len(x), 1)), x])
    predMeans = np.zeros(x.shape[0])
    predStds = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        predMeans[i], predStds[i] = getPredictiveParams(x[i,].T,
                                                        postMean,
                                                        postVar)
    predStds = np.sqrt(predStds)
    plt.plot(x[:,1], predMeans, 'b')
    plt.plot(x[:,1], predMeans + predStds, 'b--')
    plt.plot(x[:,1], predMeans - predStds, 'b--')
    plt.xticks([-1, 0, 1])
    plt.yticks([-0.5, 0, 0.5])
    plt.xlim([-1, 1])
    plt.ylim([-1, 1])
    plt.xlabel("x")
    plt.ylabel("y")

### 5.1 Implement likelihoodFunc

In [6]:
def likelihoodFunc(W, x, y_train, likelihood_var):
    '''
    Implement likelihoodFunc. This function returns the 
    data likelihood
    given f(y_train | x; W) ~ Normal(w^Tx, likelihood_var).

    Args:
        W: Weights
        x: Training design matrix with first col all 
        ones (np.matrix)
        y_train: Training response vector (np.matrix)
        likelihood_var: likelihood variance

    Returns:
        likelihood: Data likelihood (float) - Gaussian pdf
        Bagimsizlik assumptioni yaptik, carparak likelihood alacagiz
    '''

    mean = np.dot(np.transpose(W), x[i])
    ## np.prod computes the gaussian pdf of the vector with independence assumption
    likelihood = np.prod(np.array((np.exp(-((np.square(np.substract(x[i], mean))))/(2*(np.square(likelihood_var)))))/np.sqrt(2*((np.pi)*likelihood_var))))

    return likelihood

In [81]:
def generateData(dataSize, noiseParams, actual_weights):
    # x1: from [0,1) to [-1,1)
    x1 = -1 + 2 * np.random.rand(1, dataSize)
#     print ('x1 = ' + str(x1))
    # appending the bias term
    xtrain = np.matrix(np.c_[np.ones((dataSize, 1)), x1])
#     print ('xtrain = ' + str(xtrain))
    # random noise
    noise = np.matrix(np.random.normal(
                            noiseParams["mean"],
                            noiseParams["var"],
                            (dataSize, 1)))
#     print ('noise = ' + str(noise))
    ytrain = (xtrain * actual_weights) + noise

    return xtrain, ytrain

In [82]:
np.c_[np.ones((3 , 1)), [22, 33, 4]]

array([[  1.,  22.],
       [  1.,  33.],
       [  1.,   4.]])

In [83]:
actual_weights = np.random.rand(50, 1)

In [84]:
noiseParams = {"mean": 10, "var":5}

In [85]:
dataSize = 100

In [86]:
generateData(dataSize, noiseParams, actual_weights)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [64]:
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
generateData(dataSize, noiseParams, actual_weights)[1]

matrix([[ 13.72438784,  13.38298747,  12.77021872, ...,  13.03335669,
          13.70017636,  13.20091144],
        [ 11.80569195,  11.48904093,  10.92426567, ...,  11.20942195,
          11.83207583,  11.42179886],
        [ 11.84791733,  11.57344121,  11.09045076, ...,  11.41312801,
          11.9605198 ,  11.7018855 ],
        ..., 
        [  7.84451   ,   7.48819133,   6.84649327, ...,   7.09635915,
           7.78980087,   7.2368962 ],
        [ 14.12000348,  13.77134243,  13.14449391, ...,  13.40117241,
          14.08094892,  13.55557776],
        [ 10.98552349,  10.65979415,  10.07741438, ...,  10.35449412,
          10.99334846,  10.55042979]])